In [22]:
from datetime import datetime, timedelta
from pathlib import Path

from adcircpy import AdcircMesh, AdcircRun, Tides
from adcircpy.forcing.winds import BestTrackForcing
from adcircpy.utilities import download_mesh

import shapefile
import geopandas as gpd
from shapely.geometry import Point, Polygon

import numpy as np

stage = 'staging'
HURRICANE = 'Matthew2016'
OUTPUT_DIRECTORY = stage+'/runs/'+HURRICANE+'_v2b_mangrove_test'
shapefile_path = 'datasets/mangrove_covers/refined/mangrove_polygon_refined.shp'

In [ ]:
def add_mannings_n_for_polygon(mesh, shapefile_path, mannings_n_value):
    """
    Add Manning's n values to mesh nodes within polygons from a shapefile.
    
    Parameters:
    -----------
    mesh : adcircpy.AdcircMesh
        ADCIRC mesh object
    shapefile_path : str
        Path to the shapefile containing polygons
    mannings_n_value : float
        Manning's n value to apply to all nodes within the polygons
    
    Returns:
    --------
    adcircpy.AdcircMesh
        The modified mesh object with updated Manning's n values
    """
    default_n = 0.025  # Default Manning's n value

    # Initialize Manning's n if it doesn't exist

    #if not hasattr(mesh, 'mannings_n_at_sea_floor'):
        # Create an array with default values based on the number of rows in coords
     #   mesh.mannings_n_at_sea_floor = np.full(len(mesh.coords), default_n)
    
    # Load polygons from shapefile
    print(f"Loading polygons from {shapefile_path}...")
    polygons = gpd.read_file(shapefile_path)
    print(f"Loaded {len(polygons)} polygons")
    
    # Track total modified nodes
    total_modified = 0
    
    # Process each polygon
    node_values = np.array([])
    for idx, row in polygons.iterrows():
        polygon = row.geometry
        
        # Find and update nodes within this polygon
        count = 0
        if polygon is not None:
            for i, node_row in mesh.coords.iterrows():
                # Access coordinates using column names (adjust if different)
                x = node_row['x']  # Assuming column is named 'x'
                y = node_row['y']  # Assuming column is named 'y'
                
                point = Point(x, y)    
                if polygon.contains(point): # add new cover mannings
                    node_values = np.append(node_values, mannings_n_value)
                    #mesh.mannings_n_at_sea_floor[i] = mannings_n_value
                    count += 1
                else: # add default mannings
                    node_values = np.append(node_values, default_n)
        else:
            print(f"Polygon {idx} is None, skipping...")
            continue
        # Update the mesh's mannings_n_at_sea_floor array
        node_values = node_values.reshape(len(node_values), 1)
        mesh.mannings_n_at_sea_floor = node_values
        
        total_modified += count
        print(f"Polygon {idx}: Updated {count} nodes with Manning's n = {mannings_n_value}")
    
    print(f"Total nodes modified with non-default values: {total_modified}")
    return mesh

In [24]:
# open mesh file
mesh = AdcircMesh.open('datasets/meshes/ww_gonave_v2b_mangrove.grd', crs=4326)    

# Adding Manning's
# generate tau0 factor
mesh.generate_tau0()

# also add Manning's N to the domain (constant for this example)
#mesh.mannings_n_at_sea_floor = np.full(mesh.values.shape, 0.025)
mesh = add_mannings_n_for_polygon(mesh, shapefile_path, 0.2)
# initialize tidal forcing and constituents
tidal_forcing = Tides()
tidal_forcing.use_all()
mesh.add_forcing(tidal_forcing)

/home/aesgeorges/.local/lib/python3.10/site-packages/adcircpy/mesh/parsers/grd.py:22: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  output['nodes'] = pandas.read_csv(
/home/aesgeorges/.local/lib/python3.10/site-packages/adcircpy/mesh/parsers/grd.py:37: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  output['elements'] = pandas.read_csv(


Loading polygons from datasets/mangrove_covers/refined/mangrove_polygon_refined.shp...
Loaded 3 polygons
Polygon 0: Updated 2974 nodes with Manning's n = 0.2
Polygon 1 is None, skipping...
Polygon 2 is None, skipping...
Total nodes modified with non-default values: 2974


In [25]:
# set simulation dates
spinup_time = timedelta(days=15)                                                                        
duration = timedelta(days=11)
start_date = datetime(2016, 9, 28, 12)
end_date = start_date + duration

In [26]:
# initialize wind forcing
wind_forcing = BestTrackForcing(storm=HURRICANE, nws=20)
mesh.add_forcing(wind_forcing)

/home/aesgeorges/.local/lib/python3.10/site-packages/stormevents/nhc/track.py:1137: RuntimeWarning: invalid value encountered in divide
  speeds = pandas.Series(distances / abs(intervals), index=indices)


In [27]:
# instantiate driver object
driver = AdcircRun(mesh, start_date, end_date, spinup_time, netcdf=True)   
driver.NOUTGE = -3
driver.NOUTGV = -3
driver.NOUTGM = -3
driver.TOUTFGE = 30
driver.TOUTFGV = 30
driver.TOUTFGM = 30
driver.NSPOOLGE = 20
driver.NSPOOLGV = 20
driver.NSPOOLGM = 20
#driver.DTDP = 10
# write driver state to disk
driver.write(OUTPUT_DIRECTORY, overwrite=True)

/home/aesgeorges/.local/lib/python3.10/site-packages/adcircpy/mesh/fort14.py:58: FutureWarning: Assigning CRS to a GeoDataFrame without a geometry column is now deprecated and will not be supported in the future.
  self._gdf = gpd.GeoDataFrame(data, crs=self._mesh.crs)


In [15]:
bash_script = rf"""
#!/bin/bash

current_dir=$(pwd)
subdomain_dir=$current_dir/s1_gonave
folder_name=$(basename $current_dir)

mkdir ../../logs
mkdir ../../logs/$folder_name
log_header="Logs of run attempts and their parameters."
log_header2="Hurricane $folder_name"
log_header3="------------------------------------------"
log_header4="id\t\t\t\t\t\tdt\t\t\t\t\t\ttype\t\t\t\t\t\tcomment"
echo -e $log_header > runlog.out
echo -e $log_header2 >> runlog.out
echo -e $log_header3 >> runlog.out
echo -e $log_header4 >> runlog.out

cp ../../../scripts/fort22fix.py fort22fix.py
python fort22fix.py

cp ../../../scripts/run_new.sh run.sh
cp ../../../scripts/slurmcold.sh slurm.sh

current_dir=$(pwd)
folder_name=$(basename $current_dir)
sed -i "s/^#SBATCH --job-name=.*$/#SBATCH --job-name=$folder_name/" slurm.sh

# subdomain file structure setup
mkdir s1_gonave
subdomain_dir=$current_dir/s1_gonave
cp ../../../datasets/s1_gonave/shape.e14 s1_gonave/shape.e14
# remove BOM (\ufeff) from shape.e14
sed -i '1s/^\xef\xbb\xbf//' s1_gonave/shape.e14

source ~/anaconda3/etc/profile.d/conda.sh  # Adjust the path to your Conda installation
conda activate py2
python2.7 ../../../scripts/gensub.py $current_dir $subdomain_dir

chmod +x run.sh slurm.sh 

"""

# Write the bash script to the output directory
with open(OUTPUT_DIRECTORY + '/init.sh', 'w') as file:
    file.write(bash_script)

print("Bash script created and outputted at", OUTPUT_DIRECTORY)

Bash script created and outputted at staging/runs/Matthew2016_v2b_mangrove_test
